In [177]:
# Check that imports for the rest of the file work.
import tensorflow.compat.v1 as tf
!pip install tensorflow-gan
import tensorflow_gan as tfgan
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Allow matplotlib images to render immediately.
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)  # Disable noisy outputs.

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [178]:
SP500_average = pd.read_csv(
    "./data_stock/SP500_average.csv",
    ).drop([0]).drop(columns=['Date'])
SP500_average.head()


,Open,High,Low,Close
1,4366.64,4411.01,4287.11,4356.45
2,4356.32,4417.35,4222.62,4410.13
3,4471.38,4494.52,4395.34,4397.94
4,4547.35,4602.11,4477.95,4482.73
5,4588.03,4611.55,4530.20,4532.76


In [179]:
def input_fn(mode, params):
  assert 'batch_size' in params
  assert 'noise_dims' in params
  bs = params['batch_size']
  nd = params['noise_dims']
  split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'test'
  shuffle = (mode == tf.estimator.ModeKeys.TRAIN)
  just_noise = (mode == tf.estimator.ModeKeys.PREDICT)
  
  noise_ds = (tf.data.Dataset.from_tensors(0).repeat()
              .map(lambda _: tf.random.normal([bs, nd])))
  if just_noise:
    return noise_ds
  SP500_average = pd.read_csv(
    "./data_stock/SP500_average.csv",
    ).drop([0]).drop(columns=['Date'])
  SP500_average_ds = tf.data.Dataset.from_tensor_slices(SP500_average.values.tolist()).cache()
  train_size = tf.data.experimental.cardinality(SP500_average_ds)
    
  if split == 'train':
    SP500_average_ds.take(train_size)
  else:
    SP500_average_ds.skip(train_size)
    
  if shuffle:
    SP500_average_ds = SP500_average_ds.shuffle(
        buffer_size=10000, reshuffle_each_iteration=True)
  SP500_average_ds = (SP500_average_ds.batch(bs, drop_remainder=True)
               .prefetch(tf.data.experimental.AUTOTUNE))

  return tf.data.Dataset.zip((noise_ds, SP500_average_ds))

In [180]:
# sanity check dataset
params = {'batch_size': 100, 'noise_dims':64}
with tf.Graph().as_default():
  ds = input_fn(tf.estimator.ModeKeys.TRAIN, params)
  example = next(iter(tfds.as_numpy(ds)))[1]
  print(example)

[[3356.04 3363.29 3335.44 3360.47]
 [3367.27 3409.57 3367.27 3408.63]
 [3579.31 3581.23 3556.85 3557.54]
 [4367.43 4416.75 4367.43 4395.64]
 [4210.34 4238.04 4201.64 4232.6 ]
 [3352.7  3368.95 3310.47 3340.97]
 [3791.84 3843.09 3791.84 3826.31]
 [4319.57 4365.57 4290.49 4363.55]
 [4089.95 4098.19 4082.54 4097.17]
 [3411.23 3428.92 3384.45 3385.49]
 [3363.56 3402.93 3363.56 3383.54]
 [3226.14 3278.7  3209.45 3246.59]
 [3385.87 3397.18 3361.39 3380.8 ]
 [4484.4  4492.81 4482.28 4486.23]
 [3885.55 3902.92 3874.71 3876.5 ]
 [4204.78 4204.78 4164.4  4166.45]
 [3323.17 3351.03 3318.14 3349.16]
 [3438.5  3460.53 3415.34 3453.49]
 [3515.47 3527.94 3480.55 3488.67]
 [3138.7  3154.9  3127.12 3131.29]
 [4206.14 4218.78 4176.81 4211.47]
 [2514.92 2538.18 2459.96 2488.65]
 [3793.58 3851.69 3730.19 3841.94]
 [4594.96 4651.14 4583.16 4649.23]
 [4497.34 4520.4  4496.41 4519.63]
 [3288.26 3302.73 3284.53 3294.61]
 [3668.28 3682.73 3657.17 3666.72]
 [3114.4  3115.01 3032.13 3050.33]
 [4513.76 4537.36 45

In [208]:
def _dense(inputs, units, l2_weight):
  return tf.layers.dense(
      inputs, units, None,
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _dense_sigmoid(inputs, units, l2_weight):
  return tf.layers.dense(
      inputs, units, tf.nn.sigmoid,
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _batch_norm(inputs, is_training):
  return tf.layers.batch_normalization(
      inputs, momentum=0.999, epsilon=0.001, training=is_training)

def _deconv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d_transpose(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=tf.nn.relu, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _conv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=None, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))


In [209]:
def unconditional_generator(noise, mode, weight_decay=2.5e-5):
  """Generator to produce unconditional MNIST images."""
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  net = _dense(noise, 100, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 72, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 10, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 4, weight_decay)

  return net

  return net

In [210]:
_leaky_relu = lambda net: tf.nn.leaky_relu(net, alpha=0.01)

def unconditional_discriminator(price, unused_conditioning, mode, weight_decay=2.5e-5):
  del unused_conditioning
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  net = _dense(price, 100, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 72, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 10, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 1, weight_decay)

  return net

In [211]:
from tensorflow_gan.examples.mnist import util as eval_util
import os

def get_eval_metric_ops_fn(gan_model):
  real_data_logits = tf.reduce_mean(gan_model.discriminator_real_outputs)
  gen_data_logits = tf.reduce_mean(gan_model.discriminator_gen_outputs)
  return {
      'real_data_logits': tf.metrics.mean(real_data_logits),
      'gen_data_logits': tf.metrics.mean(gen_data_logits),
  }

In [212]:
train_batch_size = 32 #@param
noise_dimensions = 4 #@param
generator_lr = 0.001 #@param
discriminator_lr = 0.0002 #@param

def gen_opt():
  gstep = tf.train.get_or_create_global_step()
  base_lr = generator_lr
  # Halve the learning rate at 1000 steps.
  lr = tf.cond(gstep < 1000, lambda: base_lr, lambda: base_lr / 2.0)
  return tf.train.AdamOptimizer(lr, 0.5)

gan_estimator = tfgan.estimator.GANEstimator(
    generator_fn=unconditional_generator,
    discriminator_fn=unconditional_discriminator,
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss,
    discriminator_loss_fn=tfgan.losses.wasserstein_discriminator_loss,
    params={'batch_size': train_batch_size, 'noise_dims': noise_dimensions},
    generator_optimizer=gen_opt,
    discriminator_optimizer=tf.train.AdamOptimizer(discriminator_lr, 0.5),
    get_eval_metric_ops_fn=get_eval_metric_ops_fn)

In [ ]:
# Disable noisy output.
tf.autograph.set_verbosity(0, False)

import time
steps_per_eval = 500 #@param
max_train_steps = 5000 #@param
batches_for_eval_metrics = 100 #@param

# Used to track metrics.
steps = []
real_logits, fake_logits = [], []
real_mnist_scores, mnist_scores, frechet_distances = [], [], []

cur_step = 0
start_time = time.time()
while cur_step < max_train_steps:
  next_step = min(cur_step + steps_per_eval, max_train_steps)

  start = time.time()
  gan_estimator.train(input_fn, max_steps=next_step)
  steps_taken = next_step - cur_step
  time_taken = time.time() - start
  print('Time since start: %.2f min' % ((time.time() - start_time) / 60.0))
  print('Trained from step %i to %i in %.2f steps / sec' % (
      cur_step, next_step, steps_taken / time_taken))
  cur_step = next_step
  
  # Calculate some metrics.
  metrics = gan_estimator.evaluate(input_fn, steps=batches_for_eval_metrics)
  steps.append(cur_step)
  real_logits.append(metrics['real_data_logits'])
  fake_logits.append(metrics['gen_data_logits'])
  print('Average discriminator output on Real: %.2f  Fake: %.2f' % (
      real_logits[-1], fake_logits[-1]))
  
  # Vizualize some images.
  iterator = gan_estimator.predict(
      input_fn, hooks=[tf.train.StopAtStepHook(num_steps=21)])
  try:
    daily_prices = np.array([next(iterator) for _ in range(20)])
  except StopIteration:
    pass
  for p in daily_prices:
    print(p)
  
  

<ipython-input-208-5a6d8c6fd259>:2: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


Time since start: 0.30 min
Trained from step 0 to 500 in 28.23 steps / sec
Average discriminator output on Real: 1086.13  Fake: 54.31
[   6.4926496  149.41077    155.02751   -294.27905  ]
[ 184.20216    30.018309   19.314964 -213.68317 ]
[ 184.07515    16.113003  234.30624  -428.79193 ]
[ 239.70973    49.634525  107.92092  -347.82047 ]
[ 1.5318539e+02  3.3687103e-01  1.5161713e+02 -4.4768878e+02]
[  60.782314   39.98801    77.09361  -270.02844 ]
[ 107.667595   93.3926     78.99993  -187.05069 ]
[ 297.35608   173.09541    48.247345 -152.95764 ]
[ 170.81488   -10.375096  167.47704  -264.1674  ]
[ 283.8185   184.78732  241.46326 -384.43173]
[  63.405647   92.677734  153.28928  -310.16168 ]
[  69.933205  155.91492   147.35551  -429.38675 ]
[  42.098965  114.72638   101.60146  -266.4379  ]
[ 17.741762   2.827221  26.945404 -40.912533]
[  76.97097  117.63283  190.93979 -392.07483]
[ 134.8925     -13.9446535  125.74613   -327.77008  ]
[  78.72778    59.41582   109.83474  -103.793304]
[   6.12